In [235]:
import pandas as pd
import numpy as np
import re
import tqdm
from bs4 import BeautifulSoup
import time
import matplotlib.pyplot as plt

In [9]:
# この文字列ありますか？関数
def search_object(search,object):
    return bool(re.search(search,object))

In [ ]:
# csvファイルごとに抜き出してあとでくっつける
num_files = 1000
suumo_details = pd.DataFrame()
for num in tqdm.tqdm(range(1,num_files+1)):
    file_path = f'./soup_tank/soup_tank_{num * 100}.csv'
    suumo_hundred = pd.read_csv(file_path)

    # soupから情報を抜き出す
    data_rooms_all = []
    for url,html in zip(suumo_hundred['url'],suumo_hundred['soup']):
        search = '(<h1 class="section_h1-header-title">)(.*)(</h1>)'
        is_exist = search_object(search,html)
        if is_exist :

            soup_room = BeautifulSoup(html, 'html.parser')

            # 部屋情報の器
            data_room = []

            # 物件名
            name = soup_room.find(class_='section_h1-header-title').text
            data_room.append(name)
            # 費用に関する情報
            colname = ['家賃','管理費・共益費','敷金','礼金','保証金','敷引・償却']
            cost_info = soup_room.find_all(class_='property_view_note-list')
            for mother in cost_info:
                for child in mother.find_all('span'):
                    data_room.append(child.text)
            # 基本情報
            colname = ['住所','駅徒歩','間取り','面積','築年数','階','向き','建物種別']
            basic_info = soup_room.find_all(class_='property_view_table-body')
            for k, (name, info ) in enumerate(zip(colname,basic_info)):
                if k == 1:
                    data_room.append(info.find('div').text)
                else:
                    data_room.append(info.text)
            # オプション情報
            option = soup_room.find_all(class_='inline_list')
            data_room.append(option[1].text)
            # 概要情報
            colname = ['間取り詳細','構造','階建','築年月','損保','駐車場','入居','取引態様',
                       '条件','取り扱い店舗物件コード','SUUMO物件コード','総戸数',
 #                      '情報更新日','次回更新日','仲介手数料','保証会社','ほか初期費用'
                      ]
            overview = soup_room.find(class_='data_table table_gaiyou')
            children = overview.find_all('td')
            for name, info in zip(colname,children[:12]):
                data_room.append(info.text.strip())
            # url
            data_room.append(url)
            # 部屋情報を器に入れる
            data_rooms_all.append(data_room)
    # 100ごとに部屋情報を器に入れる
    colnames = ['物件名','家賃','管理費・共益費','敷金','礼金','保証金','敷引・償却',
                '住所','駅徒歩','間取り','面積','築年数','階','向き','建物種別',
                'オプション',
                '間取り詳細','構造','階建','築年月','損保','駐車場','入居','取引態様',
                '条件','取り扱い店舗物件コード','SUUMO物件コード','総戸数',
   #             '情報更新日','次回更新日','仲介手数料','保証会社','ほか初期費用',
                'url']
    suumo_details_tmp = pd.DataFrame(data_rooms_all,columns=colnames)
    suumo_details = pd.concat([suumo_details, suumo_details_tmp], axis=0)
    
suumo_details = suumo_details.reset_index(drop=True)


  2%|▏         | 18/1000 [07:33<7:36:40, 27.90s/it]

In [507]:
# 変数多すぎません？使わないやつ減らしましょうよ。
colnames = [
    '物件名',
    '家賃',  # 毎月かかるお金。
    '管理費・共益費',  # 物件の維持コスト。共益費は特に建物の共有部分の維持コスト。
    '敷金',  # 滞納とか契約終了時の原状復帰とかに利用される。余ったら返却される。
    '礼金',  # 物件貸し出しに対するお礼。
    '保証金',  # 基本的に敷金と同じ。礼金が含まれている場合もある。
    '敷引・償却',  # 返却されない敷金のこと。
    '住所',
    '駅徒歩',
    '間取り',
    '面積',
    '築年数',
    '階',
    '向き',
    '建物種別',
    'オプション',
    '間取り詳細',
    '構造',  # 建物の材質（木造とか）
    '階建',
    '築年月',  # 築年数で良くない?
    '損保',  # 不要?
    '駐車場',
    '入居',  # すぐ入居できますか？即 or 相談 or ??年??月?旬
    '取引態様',  # '貸主', '仲介', '代理'
    '条件',
    '取り扱い店舗物件コード',  # 不要
    'SUUMO物件コード',  # 実質ユニークキー
    '総戸数',  # 建物内の部屋の数
    'url'
]

In [ ]:
# ???万円→???
def drop_manyen(x):
    return re.sub(r'([0-9]+)万円',r'\1',x)

# ???円→???
def drop_yen(x):
    return re.sub(r'([0-9]+)円',r'\1',x)

# 文字+数字+文字 → 数字
def objnumobj_num(x):
    return re.sub(r'\w+?([0-9]+)\w+',r'\1',x)

# 最後の数字列を取り出す
def lastnum(x):
    return re.findall('[0-9]+',x)[-1]

# (文字)数字を取り出す
def get_objnum(x):
    return re.search(r'([A-Z]?)([0-9]+)',x).group()

# 住所→緯度経度変換装置
import requests
import urllib

def translate_address_coordinates(address):
    makeUrl = "https://msearch.gsi.go.jp/address-search/AddressSearch?q="
    s_quote = urllib.parse.quote(address)
    response = requests.get(makeUrl + s_quote)
    return response.json()[0]["geometry"]["coordinates"]

# 座標から距離を計算
def distance(keido1,ido1,keido2,ido2):
    import math
    # 出力は[m]
    r=6378137
    keido1 = math.radians(keido1)
    ido1 = math.radians(ido1)
    keido2 = math.radians(keido2)
    ido2 = math.radians(ido2)
    distance = r*np.arccos(round(np.sin(ido1)*np.sin(ido2)+np.cos(ido1)*np.cos(ido2)*np.cos(keido2-keido1),10))
    return distance

In [ ]:
# 家賃変数

# ??.?万円　→　??.?(float)
suumo_details['家賃'] = suumo_details['家賃'].map(lambda x:drop_manyen(x)).astype(float)

In [ ]:
# 管理費・共益費変数

# (管理費・共益費)を除去
def process_kannrihi_Remove_unnecessary_characters(x):
    return re.search('(管理費・共益費:\xa0)(.+)',x).group(2)
suumo_details['管理費・共益費'] = suumo_details['管理費・共益費'].map(lambda x: process_kannrihi_Remove_unnecessary_characters(x))

# 「-」は0円ということにする
suumo_details['管理費・共益費'] = suumo_details['管理費・共益費'].replace('-','0円')

# 円を除去
suumo_details['管理費・共益費'] = suumo_details['管理費・共益費'].map(lambda x: drop_yen(x))

# int型にする
suumo_details['管理費・共益費'] = suumo_details['管理費・共益費'].astype(int)

In [ ]:
# 敷金変数

# (敷金)を除去
def process_shikikin_Remove_unnecessary_characters(x):
    return re.search('(敷金:\xa0)(.+)',x).group(2)
suumo_details['敷金'] = suumo_details['敷金'].map(lambda x: process_shikikin_Remove_unnecessary_characters(x))

# 「-」は0円ということにする
suumo_details['敷金'] = suumo_details['敷金'].replace('-','0万円')

# 万円を除去
suumo_details['敷金'] = suumo_details['敷金'].map(lambda x: drop_manyen(x))

# float型にする
suumo_details['敷金'] = suumo_details['敷金'].astype(float)

In [ ]:
# 礼金変数

# (礼金)を除去
def process_reikin_Remove_unnecessary_characters(x):
    return re.search('(礼金:\xa0)(.+)',x).group(2)
suumo_details['礼金'] = suumo_details['礼金'].map(lambda x: process_reikin_Remove_unnecessary_characters(x))

# 「-」は0円ということにする
suumo_details['礼金'] = suumo_details['礼金'].replace('-','0万円')

# 万円を除去
suumo_details['礼金'] = suumo_details['礼金'].map(lambda x: drop_manyen(x))

# float型にする
suumo_details['礼金'] = suumo_details['礼金'].astype(float)

In [ ]:
# 保証金変数

# (保証金)を除去
def process_hosyokin_Remove_unnecessary_characters(x):
    return re.search('(保証金:\xa0)(.+)',x).group(2)
suumo_details['保証金'] = suumo_details['保証金'].map(lambda x: process_hosyokin_Remove_unnecessary_characters(x))

# 「-」は0円ということにする
suumo_details['保証金'] = suumo_details['保証金'].replace('-','0万円')

# 万円を除去
suumo_details['保証金'] = suumo_details['保証金'].map(lambda x: drop_manyen(x))

# float型にする
suumo_details['保証金'] = suumo_details['保証金'].astype(float)

In [ ]:
# 敷引・償却変数

# (敷引・償却)を除去
def process_shikihiki_Remove_unnecessary_characters(x):
    return re.search('(敷引・償却:\xa0)(.+)',x).group(2)
suumo_details['敷引・償却'] = suumo_details['敷引・償却'].map(lambda x: process_shikihiki_Remove_unnecessary_characters(x))

# 「-」は0円ということにする
suumo_details['敷引・償却'] = suumo_details['敷引・償却'].replace('-','0万円')

# 万円を除去
suumo_details['敷引・償却'] = suumo_details['敷引・償却'].map(lambda x: drop_manyen(x))

# float型にする
suumo_details['敷引・償却'] = suumo_details['敷引・償却'].astype(float)

In [ ]:
# 住所変数

# ～都,～区,～ に分けたい
# ついでに[都道府県][市区町村][以下]に対応したい

# 住所を(都道府県)(市区町村)(市区町村以下)に分けて取り出す
def split_address(x):
    a,b,c = re.search('(...??[都道府県])(.+?[市区町村])(.+)',x).groups()
    return a,b,c

# それぞれを変数に入れる
suumo_details[['都道府県','市区町村','市区町村以下']] = suumo_details.apply(lambda x:split_address(x.住所),axis=1,result_type='expand')

In [ ]:
# 住所のuniqueから緯度経度を取得
# csvに変換しておいたので2回目は不要　./data/csv/coordinates_details.csvを呼び出そう

all_addresses = {}
running_times = []
i = 0

for address in suumo_details['住所'].unique():
    
    start = time.time()

    coordinate = translate_address_coordinates(address)
    all_addresses[address] = coordinate
    time.sleep(1)
    
    finish = time.time()
    
    running_time = finish - start
    running_times.append(running_time)

    all_count = suumo_details['住所'].unique().shape[0]
    print(f'{i}件目：{running_time}秒')
    # 作業進捗
    complete_ratio = round(i/all_count*100,3)
    print(f'完了：{complete_ratio}%')
    # 作業の残り時間目安を表示
    running_mean = np.mean(running_times)
    running_required_time = running_mean * (all_count - i)
    hour = int(running_required_time/3600)
    minute = int((running_required_time%3600)/60)
    second = int(running_required_time%60)
    print(f'残り時間：{hour}時間{minute}分{second}秒\n')
    i += 1

In [ ]:
# 住所と緯度経度のDataFrame
# これもcsvに保存済なので2回目は不要

coordinates = pd.DataFrame(all_addresses).T
coordinates.reset_index(inplace=True)
coordinates.rename(columns={
    'index':'住所',
    0:'経度',
    1:'緯度'
}, inplace=True)
coordinates.to_csv('./data/csv/coordinates_details.csv',index=False)

In [ ]:
# 元データとくっつける
# 連打すると増殖するので注意
coordinates_details = pd.read_csv('./data/csv/coordinates_details.csv')
suumo_details = pd.merge(suumo_details, coordinates_details, on='住所', how='left')

In [ ]:
# 最寄り駅変数（正規表現）

# 最寄り駅から路線と駅を取り出す
suumo_details[['路線','駅']] = suumo_details.apply(lambda x:re.search('(.+)/(.+?)\s(.+)',x.駅徒歩).groups()[0:2],axis=1,result_type='expand')

def how_to_station():
    # 移動手段は3パターン
    how_to_station = ['歩','バス','車']
    for by in how_to_station:
        # 移動手段変数を生成
        suumo_details[f'{by}'] = np.zeros(suumo_details.shape[0])
        # その移動手段があるなら移動手段変数に格納する
        condition = suumo_details['駅徒歩'].map(lambda x: search_object(f'{by}[0-9]+分',x))
        suumo_details.loc[condition,f'{by}'] = suumo_details.loc[condition,'駅徒歩'].map(lambda x: re.search(f'{by}([0-9]+)分',x).group(1))
        # 所要時間をint型に変換
        suumo_details[f'{by}'] = suumo_details[f'{by}'].astype(int)

# 移動手段ごとに所用時間変数を作る
how_to_station()

In [ ]:
# 駅の緯度経度情報を結合

# 駅情報を読み込み
station_coordinates = pd.read_csv('./data/csv/station_coordinates.csv')
# 名前が被らないように修正
station_coordinates.rename(columns={
    '住所':'住所(駅)',
    '緯度':'緯度(駅)',
    '経度':'経度(駅)'
}, inplace=True)
# 〇〇駅に名称を変換
station_coordinates['駅'] = station_coordinates['駅'].map(lambda x: x+'駅')

# ヶはケに変える
condition = suumo_details['駅'].map(lambda x: search_object('ヶ',x))
suumo_details.loc[condition,'駅'] = suumo_details.loc[condition,'駅'].map(lambda x: re.sub('ヶ','ケ',x))
# 結合
suumo_details = pd.merge(suumo_details, station_coordinates, on='駅', how='left')

In [ ]:
# 最寄駅からの距離変数を生成

# 最寄駅からの距離[m]
suumo_details['最寄駅からの距離'] = 0
condition = ~suumo_details['経度(駅)'].isnull()
suumo_details.loc[condition,'最寄駅からの距離'] = suumo_details[condition].apply(lambda x: distance(x['経度'], x['緯度'], x['経度(駅)'], x['緯度(駅)']), axis=1)

In [ ]:
# 皇居からの距離変数を生成

# 皇居の座標を獲得
koukyo = '東京都千代田区千代田1'
keido_k, ido_k = translate_address_coordinates(koukyo)

# 皇居(都心？)からの距離[m]
suumo_details['皇居からの距離'] = 0
suumo_details['皇居からの距離'] = suumo_details.apply(lambda x: distance(x['経度'], x['緯度'], keido_k, ido_k), axis=1)

In [ ]:
# 間取り変数の変換

# ワンルームは1部屋ってことにする（かなり諸説）
condition = suumo_details['間取り'] == 'ワンルーム'
suumo_details.loc[condition,'間取り'] = '1'

# 数字部分を取り出して、部屋数変数にする
suumo_details['部屋数'] = suumo_details['間取り'].map(lambda x: re.search('([0-9]+)(.*)',x).group(1)).astype(int)

# S,L,D,K はあれば1でなければ0になる
list_alpha = ['S','L','D','K']
for alpha in list_alpha:
    suumo_details[f'間取り_{alpha}'] = 0
    condition = suumo_tokyo['間取り'].map(lambda x: search_object(alpha,x))
    suumo_details.loc[condition,f'間取り_{alpha}'] = 1

In [ ]:
# 面積変数

# ??.??m2 → ??.??
def process_area(x):
    return re.search('(.+)(m2)',x).group(1)
suumo_details['面積'] = suumo_details['面積'].map(lambda x: process_area(x))

# float型に変換
suumo_details['面積'] = suumo_details['面積'].astype('float')

In [ ]:
# 築年数変数

# 新築は築0年ということにする
suumo_details.loc[suumo_details['築年数']=='新築','築年数'] = '築0年'

# 築??年(以上)→??
# 99階以上は99階に圧縮されます
suumo_details['築年数'] = suumo_details['築年数'].map(lambda x: objnumobj_num(x))

# 整数型に変更
suumo_details['築年数'] = suumo_details['築年数'].astype(int)

In [ ]:
# 階変数

# - は1階ということにする
suumo_details.loc[suumo_tokyo['階']=='-','階'] = '1階'

# 階の前処理 -があればその後ろ、なければ(文字)数字
def floor(x):
    return search_object('-',x)*lastnum(x)+(1-search_object('-',x))*get_objnum(x)
# B? → -?,Bがなければそのまま
def basement_floor(x):
    return search_object('B',x)*('-'+re.search(r'[0-9]+',x).group())+(1-search_object('B',x))*(x)
# M? → ?.5 Mがなければそのまま
def middle_floor(x):
    return search_object('M',x)*str((int(re.search(r'[0-9]+',x).group())-0.5))+(1-search_object('M',x))*(x)
   
# 上の処理を適用する
suumo_details['階'] = suumo_details['階'].map(lambda x: floor(x)).map(lambda x: basement_floor(x)).map(lambda x: middle_floor(x))

# float型に変換
suumo_details['階'] = suumo_details['階'].astype(float)

In [ ]:
# 向き変数は処理不要
# 「-」もそのままにする：そこそこ数がある＋正解らしい処理も見当たらない

In [ ]:
# 建物種別変数は処理不要

In [ ]:
# 間取り詳細は不採用
# 処理難度が高い気がする

In [ ]:
# 構造変数は処理不要

In [ ]:
# 階建変数→階数変数
suumo_details['階数'] = suumo_details['階建'].map(lambda x: re.search('(.+)/(.+)',x).group(2))

In [ ]:
# 地下変数

# 平屋は1階建ということにする
suumo_details.loc[suumo_details['階数']=='平屋','階数'] = '1階建'

# 地下何階までありますか？関数
def underground(x):
    return search_object('地下',x)*objnumobj_num(x)+(1-search_object('地下',x))*'0'
    
# 地下がなければ０、あるなら何階まであるか
suumo_details['地下'] = suumo_details['階数'].map(lambda x: underground(x))

# int型に変換
suumo_details['地下'] = suumo_details['地下'].astype('int')

In [ ]:
# 地上変数(正規表現)

# 平屋は1階建ということにする
suumo_details.loc[suumo_details['階数']=='平屋','階数'] = '1階建'

# ??階建→??
suumo_details['地上'] = suumo_details['階数'].map(lambda x: lastnum(x))

# int型に変換
suumo_details['地上'] = suumo_details['地上'].astype(int)

In [ ]:
# 階数変数は地下地上の合算にしておく
suumo_details['階数'] = suumo_details['地上'] + suumo_details['地下']

In [ ]:
# 築年月変数いる？築年数変数で良くね？

In [ ]:
# 損保は…まあええか

In [ ]:
# 駐車場変数
# 機械式駐ってのはタワーパーキングのこと
# 駐車場あるorない + 駐車場までの距離 + 駐車場の料金 でどう？
# オプション情報はなくなります…
# 近隣 → 1 + ??[m] + ??[円]
# 敷地内 → 1 + 0[m] + ??[円]
# - → 0 + 0 + 0
# その他　→ 1 + 0[m] + 0[円]
# 問題点は、駐車場がない皆さんの0[m]や0[円]は家賃安いはずなんだけど
# 駐車場がある皆さんは0[m]や0[円]が高いはずってこと

# いったん駐車場がないってことにする
suumo_details[['駐車場有無','駐車場距離','駐車場料金']] = 0,0,0

# 「-」じゃない皆さんには駐車場が与えられます。
condition = suumo_details['駐車場'] == '-'
suumo_details.loc[~condition,'駐車場有無'] = 1

# 近隣の皆さん
condition = suumo_details['駐車場'].map(lambda x: search_object('近隣',x))
# 近隣の皆さんの駐車場距離
def difference_parking(x):
    return re.search('.+?([0-9]+)m([0-9]+)円',x).group(1)
suumo_details.loc[condition,'駐車場距離'] = suumo_details.loc[condition,'駐車場'].map(lambda x: difference_parking(x))
# 近隣の皆さんの駐車場料金
def difference_cost_near_house(x):
    return re.search('.+?([0-9]+)m([0-9]+)円',x).group(2)
suumo_details.loc[condition,'駐車場料金'] = suumo_details.loc[condition,'駐車場'].map(lambda x: difference_cost_near_house(x))

# 敷地内の皆さん
condition = suumo_details['駐車場'].map(lambda x: search_object('敷地内',x))
# 敷地内の皆さんは駐車場料金が発生しません
# 敷地内の皆さんの駐車場料金
def difference_cost_in_house(x):
    return re.search('.+?([0-9]+)円',x).group(1)
suumo_details.loc[condition,'駐車場料金'] = suumo_details.loc[condition,'駐車場'].map(lambda x: difference_cost_in_house(x))

# データ型の変換
suumo_details['駐車場距離'] = suumo_details['駐車場距離'].astype(int)
suumo_details['駐車場料金'] = suumo_details['駐車場料金'].astype(int)

In [ ]:
# 入居変数
# ??年??月??旬 → 待ち に変えるよ
soku = suumo_details['入居'] == '即'
soudan = suumo_details['入居'] == '相談'
suumo_details.loc[(~soku)&(~soudan),'入居'] = '待ち'

In [ ]:
# 取引態様は処理不要

In [ ]:
# 条件変数
# '二人入居可','子供可','ルームシェア不可','単身者可','ペット相談','ルームシェア相談','事務所利用不可','単身者限定','子供不可','楽器相談','フリーレント1ヶ月 (条件),'学生希望','事務所利用相談',
# 使うかどうかは諸説。使うなら使うで取り出しは簡単
# フリーレントとは：入居開始から数カ月家賃無料になる。だいたい条件がある。

In [428]:
# 条件変数の中身
condition_list = []
for i in range(suumo_details.shape[0]):
    condition_list_room = suumo_details.loc[i,'条件'].split('/')
    for cond in condition_list_room:
        condition_list.append(cond)
pd.DataFrame(condition_list)[0].unique().tolist()

['-',
 '二人入居可',
 '子供可',
 'ルームシェア不可',
 '単身者可',
 'ペット相談',
 'ルームシェア相談',
 '事務所利用不可',
 '単身者限定',
 '子供不可',
 '楽器相談',
 'フリーレント1ヶ月 10月契約開始まで',
 '学生希望',
 '事務所利用相談',
 'フリーレント1ヶ月',
 'フリーレント1ヶ月 家賃無料',
 'フリーレント1ヶ月 短期解約違約金あり',
 'フリーレント2ヶ月 ※詳細はお問い合わせくださいませ',
 'フリーレント2ヶ月',
 'フリーレント2ヶ月 ※1年以内解約時賃料等2か月分違約金有',
 'フリーレント2ヶ月 1年以内解約時賃料等2か月分違約金有',
 'フリーレント1ヶ月 2022年12月までの申込',
 'フリーレント2ヶ月 ※短期解約違約金有',
 'フリーレント1ヶ月 ※短期解約違約金有',
 'フリーレント2ヶ月 1年未満解約時違約金あり',
 'フリーレント1ヶ月 詳細はお問合せ下さいませ',
 'フリーレント1ヶ月 1年未満解約時違約金有',
 'フリーレント1ヶ月 １年未満の解約は短期解約違約金あり',
 'フリーレント1ヶ月 フリーレント一か月',
 'フリーレント1ヶ月 １年未満解約違約金有',
 'フリーレント1ヶ月 2022年10月31日までの申込みに適用します。',
 'フリーレント2ヶ月 10月末日まで、1年未満解約時違約金有',
 'フリーレント1ヶ月 1年未満解約時違約金：総賃料の1ヶ月分+フリーレント相当額',
 'フリーレント2ヶ月 2ヶ月',
 'フリーレント1ヶ月 詳細はお問い合わせください。',
 'フリーレント1ヶ月 1ヶ月',
 'フリーレント1ヶ月 1年未満解約時賃料等1ヶ月分の違約金有']

In [ ]:
# 取り扱い店舗物件コードはよくわからなかった

In [ ]:
# SUUMO物件コードは実質的なユニークキーとして使える？

In [449]:
# 総戸数変数
# 「-」の扱いに困る
condition = suumo_details['総戸数'] == '-'
suumo_details.loc[condition,'url'].tolist()

['https://suumo.jp/chintai/jnc_000046696863/?bc=100294945613',
 'https://suumo.jp/chintai/jnc_000077689180/?bc=100302064363',
 'https://suumo.jp/chintai/jnc_000077692991/?bc=100302075201',
 'https://suumo.jp/chintai/jnc_000077594917/?bc=100301504163',
 'https://suumo.jp/chintai/jnc_000077702981/?bc=100298383398',
 'https://suumo.jp/chintai/jnc_000077560890/?bc=100299735569',
 'https://suumo.jp/chintai/jnc_000077610602/?bc=100300085945',
 'https://suumo.jp/chintai/jnc_000077284341/?bc=100302017244',
 'https://suumo.jp/chintai/jnc_000077313315/?bc=100299794570',
 'https://suumo.jp/chintai/jnc_000077689183/?bc=100302056578',
 'https://suumo.jp/chintai/jnc_000077693008/?bc=100302075083',
 'https://suumo.jp/chintai/jnc_000077686230/?bc=100301998424',
 'https://suumo.jp/chintai/jnc_000077429070/?bc=100300476017',
 'https://suumo.jp/chintai/jnc_000077638638/?bc=100302061950',
 'https://suumo.jp/chintai/jnc_000037143890/?bc=100296426755',
 'https://suumo.jp/chintai/jnc_000077560884/?bc=1003012

In [501]:
# めぼしいオプションは？
# 他の項目見ればよくね？ってのは基本排除した
# オプションの項目にこれだけ押し込むのか…
options = [
    'バストイレ別','室内洗濯置','エアコン','システムキッチン','オートロック',
    '浴室乾燥機','洗面所独立','温水洗浄便座','TVインターホン','バルコニー',
    '宅配ボックス','都市ガス','フローリング','シューズボックス','クロゼット',
    '駐輪場','エレベーター','敷地内ゴミ置き場','防犯カメラ','ガスコンロ対応',
    '保証人不要','追炊機能浴室','洗面化粧台','脱衣所','前居室フローリング',
    '2口コンロ','初期費用カード決済可','光ファイバー','BS・CS','24時間換気システム',
    '角住戸','ネット使用料不要','グリル付','3口以上コンロ','ペット相談','24時間ゴミ出し可',
    '外壁タイル張り','CATV ','CS','ディンプルキー','ダブルロックキー','保証会社利用可',
    'オートバス','高速ネット対応','洗面所にドア','二人入居相談','全居室洋室',
    'バイク置場','2面採光','シャワー付洗面台','陽当り良好','照明付','ネット専用回線',
    '南面バルコニー','ウォークインクロゼット','駅まで平坦','耐震構造','通風良好',
    '対面式キッチン','全居室収納','日勤管理','閑静な住宅地','玄関収納','ガスレンジ付',
    '人感照明センサー','南面リビング','エアコン全室','平坦地','LDK12畳以上','耐火構造',
    'デザイナーズ','クロゼット2ヶ所','眺望良好','独立型キッチン','フリーレント','LAN',
    '分譲賃貸','セキュリティ会社加入済','IHクッキングヒーター','最上階','高層階','未入居',
    '家賃カード決済可','カードキー','室内物干機','楽器相談','タワー型マンション',
    'CATVインターネット','学生相談','24時間緊急通報システム','シューズWIC','浄水器',
    '複層ガラス','床暖房','シャッター','始発駅','フロントサービス','緑豊かな住宅地',
    'オープンキッチン','駅前','ペット専用設備','食器洗乾燥機','ワイドバルコニー',
    '2WAYバルコニー','テラス','2面バルコニー','和室','ディスポーザー','プロパンガス',
    'オール電化','家具付','ルーフバルコニー','ロフト','','',
]
option_list = []
for i in range(suumo_details.shape[0]):
    option_room = suumo_details.loc[i,'オプション'].strip().split('、')
    for option in option_room:
        option_list.append(option)
pd.DataFrame(option_list)[0].value_counts()[300:350]

LDK18畳以上     1
2階洗面台        1
クッションフロア     1
テレビ          1
ハイサッシ        1
バリアフリー       1
AED付         1
路面電車沿線       1
内装リフォーム後渡    1
Name: 0, dtype: int64

In [ ]:
suumo_details.to_csv('./data/csv/SuumoDetails20221031.csv',index=False)